# CLI

In [ ]:
# these are needed for the docs site
from IPython import get_ipython

The `jupyter lite` (or `jupyter-lite`) CLI provides tools for lifecycle of combining...
- **user-authored content** like _Notebooks_ and _Lab Extensions_.
- the core JupyterLite **static assets** 

... into a **ready-to-[deploy](./deploying.md) site** and **[reproducible](#Reproducibility) archives**.

## Installation

In [ ]:
!pip install jupyterlite
!jupyter lite --version

### Extras

Some extra features of different _addons_ have additional dependencies.

```bash
pip install jupyterlite[contents]  # jupyter_server for better contents API
pip install jupyterlite[serve]  # tornado for better local preview
pip install jupyterlite[lab]  # a known-compatible jupyterlab (entails `contents` and `serve`)
# TODO: [archive]  # use libarchive
```

## The Lite Dir

When you run `jupyter lite` commands, it assumes your current working directory is the partial contents of a JupyterLite site. You can override this with `--lite-dir`. By default, the built site will be created in `_output`, but can be overridden with `--output-dir`.

In [ ]:
import os, pathlib, tempfile

if "TMP_DIR" not in globals():
    TMP_DIR = pathlib.Path(tempfile.mkdtemp(prefix="_my_lite_dir_"))
os.chdir(TMP_DIR)
pathlib.Path.cwd()

### Well-known Files

Some files in your `--lite-dir` that have special meaning:

| paths                       | file                | if found                                       |
|-----------------------------|---------------------|------------------------------------------------|
| `.`<br/>`./lab`<br/>`./retro` | `jupyter-lite.json` | merge contents with static in `_output/{path}/jupyter-lite.json` |
| `.`<br/>`./lab`<br/>`./retro` | `overrides.json`    | merge with static `_output/{*}/jupyter-lite.json`          ||
| `./files/`                    | `*`                 | copy verbatim to `_output/files/*` and index in `/api/contents` |

## Usage

The CLI provides its own documentation, under `--help` (or `-h`)

In [ ]:
!jupyter lite --help

### Status

This command is always safe to run, and provides an overview of what JupyterLite has been up to.

In [ ]:
!jupyter lite status

### List

This command is always safe to run, and provides an overview of what JupyterLite _might_ do.

> _TODO: improve on default output_

In [ ]:
!jupyter lite list

### Init

This copies all the static data to the `--output-dir`.

In [ ]:
!jupyter lite init

### Build

This copies all the **user-authored content** to the `--output-dir`, and applies appropriate changes to e.g. generated Contents API responses.

In [ ]:
!jupyter lite build

### Check

Use all available mechanisms to verify that the build folder conforms to schema, etc.

In [ ]:
!jupyter lite check

### Archive

Turn the _output directory_ into `.tgz` file. This is usually easier to move around than (sometimes) hundreds of files, and can be used as the baseline for future sites.

In [ ]:
!jupyter lite archive

But let's talk about a more _reproducible_ asset.

In [ ]:
import shutil
shutil.rmtree(TMP_DIR / "_output")

#### Reproducibility

> _🛠️ This feature is a **work-in-progress**, and should not be relied upon by any production workflows **Just Yet**._

If the `--source-date-epoch` is given, a number of measures will be taken to *try to ensure* that the output of `jupyter lite archive`, an npm-compatible `tgz` package, always returns a bit-for-bit [reproducible build](https://reproducible-builds.org).

The most obvious change is that the modified time of each file "clamped" to that time. Some other changes:
- file ownership will be reset
- predictable sorting will be used
- additional checks will be applied

```{note}
This is a shortcut for setting the environment variable `SOURCE_DATE_EPOCH`:

| platform         | command                                               |
|------------------|-------------------------------------------------------|
| Linux<br/>MacOS  | `export SOURCE_DATE_EPOCH=<a timestamp>`              |
| Windows          | `set SOURCE_DATE_EPOCH=<a timestamp>`                 |
| Python           | `os.environ.update(SOURCE_DATE_EPOCH, <a timestamp>)` |
```

In [ ]:
if not "source_date_epoch" in globals():
    from datetime import datetime
    source_date_epoch = int(datetime.utcnow().timestamp())
    
print("SOURCE_DATE_EPOCH is", source_date_epoch)

In [ ]:
!jupyter lite archive --source-date-epoch {source_date_epoch} --output-archive ./a.tgz

If we clear out our `_output`, and rebuild, we should always get the same file.

In [ ]:
import shutil
shutil.rmtree(TMP_DIR / "_output")
!jupyter lite archive --source-date-epoch {source_date_epoch} --output-archive ./b.tgz

In [ ]:
from hashlib import sha256
a, b = [sha256((TMP_DIR / f"{x}.tgz").read_bytes()).hexdigest() for x in "ab"]
print("We built app archives with the SHA256SUMS of:\n", a, "\n", b)
try:
    assert a == b, "We did not reproducibly build today.\n- {}\n- {}\n\n".format(a, b)
except AssertionError as err:
    if shutil.which("diffoscope"):
        print("We did NOT reproducibly build today, checking in with `diffoscope`...")
        !diffoscope a.tgz b.tgz
    print("...but at least we tried REALY hard!\n")